In [28]:
import tensorflow as tf
from keras.src.backend.jax.nn import sparse_categorical_crossentropy
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16
import tensorflow_datasets as tfds
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
import numpy as np
import os



## Use Cats & Dogs Dataset Load a subset of the data
##Create random bounding box in each image
## Define a model that is able to do both:
## 1. Classify the image either cat or dog
## 2. Predict the bounding box that was generated in the begging

### setup

In [29]:


# path to images
data_dir = "data/train/"

img_size = 128
num_classes = 2
images = []# hold image tensors after loading
labels = [] # hold corresponding labels for each list

for fname in os.listdir(data_dir):
    if fname.endswith(".jpg"):
        img_path = os.path.join(data_dir, fname)
        #load and resize image
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (img_size, img_size))
        img = tf.cast(img, tf.float32)

        images.append(img)

        # assing label based on filename
        if "cat" in fname.lower():
            labels.append(0)
        elif "dog" in fname.lower():
            labels.append(1)
        else:
            continue # skips it if not clear


# converts to tensors

images = tf.stack(images)
labels = tf.convert_to_tensor(labels, dtype=tf.int32)







### Create a random bounding box in each image

In [30]:
#to create t random bounding box you need to pick two random x and y and it starts with minium x and then starts with min y then max x nad max y

def preprocess_bound_box(image):
    #resizes and normalize the image
    image = tf.image.resize(image, (img_size, img_size))
    image = tf.cast(image, tf.float32) / 255.0 # divide by 255.0 to normalize pixel values
    #pixel values range 0,255 for each color channel
    # Nueral network work better with smaller ranges

    #Generate random bounding box in nroamlized coordinates
    #Ensure   x_min < x_max and y_min < y_max
#picks random float in the range
    x_min = np.random.uniform(0, 0.5) #0, 0.5 to make sure box starts somewhree top left half
    y_min = np.random.uniform(0, 0.5)

    x_max = np.random.uniform(0.5, 1.0)
    y_max = np.random.uniform(0.5, 1.0)

    boundbox = np.array([x_min, y_min, x_max, y_max], dtype=np.float32)

    return image, boundbox

#ex usage with list of images

processed_images = []
boundboxes = []

for img in images:
    img_proc, boundbox = preprocess_bound_box(img)
    processed_images.append(img_proc)
    boundboxes.append(boundbox)

processed_images = tf.stack(processed_images)
boundboxes = tf.stack(boundboxes)

print("Processed images shape", processed_images.shape)
print("Boundboxes shape", boundboxes.shape)
print("Example Bound box:", boundboxes[0].numpy())





Processed images shape (4097, 128, 128, 3)
Boundboxes shape (4097, 4)
Example Bound box: [0.49761626 0.35733223 0.90809345 0.7990993 ]


#### Interpreation

Processed images shape (4097, 128, 128, 3)
4097 is amount of images  in dataset
129, 128 is height and width
3 is number of colour channels

Boundboxes shape (4097, 4)
4097 number of bounding boxes her image (1 per image)
4 each bounding box is defined by 4 coordinates

Example Bound box: [0.43754366 0.26004314 0.65264034 0.5635852 ]
x_min, y_min, x_max, y_max
left edge of the box (43.7%)
top edge of the box (26%)


Model to clasisify image and predict the bounding box that was genereated
- Using vgg16 in this easy case

In [31]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model

IMG_SIZE = 128
NUM_CLASSES = 2

# 1. Input layer
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# 2. Base VGG16
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_tensor=inputs   # <-- connect directly to your inputs
)
base_model.trainable = False

# 3. Flatten & dense layers
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)

# 4. Multi-task outputs
class_output = layers.Dense(NUM_CLASSES, activation='softmax', name='class_output')(x)
bbox_output = layers.Dense(4, activation='sigmoid', name='bbox_output')(x)

# 5. Model
model = Model(inputs=inputs, outputs=[class_output, bbox_output])
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 128, 128,  │      1,792 │ input_layer_3[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 128, 128,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 64, 64,    │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 64, 64,    │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 64, 64,    │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 32, 32,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 32, 32,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 32, 32,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 32, 32,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 16, 16,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 16, 16,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 16, 16,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 16, 16,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 8, 8, 512) │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 8, 8, 512) │  2,359,808 │ block4_pool[0][0] │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv2        │ (None, 8, 8, 512) │  2,359,808 │ block5_conv1[0][

 Total params: 18,912,582 (72.15 MB)

 Trainable params: 4,197,894 (16.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

compile

In [32]:

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss={
        "class_output": "sparse_categorical_crossentropy",
        "bbox_output": "mse"

    },
    loss_weights={
        "class_output": 0.2, # in each back propgation it does 20% of the effect
        "bbox_output": 0.8, # this one holds 80%
    },
    metrics={
        "class_output":"accuracy",
        "bbox_output":"mse"
    }
)



prep dataset

In [33]:
dataset = tf.data.Dataset.from_tensor_slices(
    (processed_images, {'class_output': labels, 'bbox_output': boundboxes})
)
dataset = dataset.shuffle(1000).batch(16).prefetch(tf.data.AUTOTUNE)

train

In [34]:
history = model.fit(
    dataset,
    epochs=10,
)

Epoch 1/10
257/257 ━━━━━━━━━━━━━━━━━━━━ 124s 454ms/step - bbox_output_loss: 0.0306 - bbox_output_mse: 0.0305 - class_output_accuracy: 0.9165 - class_output_loss: 0.1994 - loss: 0.0645
Epoch 2/10
  2/257 ━━━━━━━━━━━━━━━━━━━━ 1:48 427ms/step - bbox_output_loss: 0.0282 - bbox_output_mse: 0.0282 - class_output_accuracy: 0.8594 - class_output_loss: 0.3599 - loss: 0.0945

KeyboardInterrupt: 

predict

In [ ]:
sample_img = processed_images[0:1]
class_pred, bbox_pred = model.predicts(sample_img)

print("Class probabilities:", class_pred)
print("Predicted bounding box (normalized):", bbox_pred)